# Autonaming Component cells

Problem:

1. In GDS different cells must have different names. Relying on the incrementals
naming convention can be dangerous when you merge masks that have different
cells build at different run times or if you merge masks with other tools like Klayout.
2. In GDS two cells cannot have the same name.

Solution: The decorator `pp.cell` fixes both issues:

1. By giving the cell a unique name depending on the parameters that you pass
2. By creating a cache of cells where we use the cell name as the key. Avoiding to create two cells with the same name during the same python runtime

Lets see how it works

In [ ]:
import pp


@pp.cell
def wg(length=10, width=1):
    c = pp.Component()
    c.add_polygon([(0, 0), (length, 0), (length, width), (0, width)], layer=(1, 0))
    c.add_port(name="W0", midpoint=[0, width / 2], width=width, orientation=180)
    c.add_port(name="E0", midpoint=[length, width / 2], width=width, orientation=0)
    return c

See how the cells get the name from the parameters that you pass them

In [ ]:
c = wg()
print(c)

c = wg(width=0.5)
print(c)

How can you have add two different references to a cell with the same parameters?

In [ ]:
# Problem
import pp

c = pp.Component('problem')
R1 = pp.c.rectangle(size=(4,2))  # Creates a rectangle (same Unique ID uid)
R2 = pp.c.rectangle(size=(4,2))
# Try Create a new rectangle that we want to change (but has the same name so we will get R1 from the cache)

r1r = c << R1  # Add the first rectangle to c
r2r = c << R2  # Add the second rectangle to c
c

In [ ]:
print(R1 == R2)
print(R1)
print(R2)

# But now I want to rotate R2 -- I can't because it doesn't exist! The error is that it rotates both references!
R2.rotate(45)
# I think I'm rotating a second rectangle, but actually R2 points to R1 even though I specifically tried to create two rectangles
pp.qp(c)

if you run the cell above you will see the cell rotating. This is a bad way to manipulate cells. That's why wrote the references tutorial.

In [ ]:
# Solution: use references
import pp

c = pp.Component('solution')
R = pp.c.rectangle(size=(4,2))  

r1 = c << R  # Add the first rectangle reference to c
r2 = c << R  # Add the second rectangle reference to c

r2.rotate(45)
print(c)
pp.qp(c)

# Adding port markers

In [ ]:
import pp
c = pp.c.waveguide()
c

In [ ]:
c2 = pp.add_pins_container(c)
c2

We can even define the `pins_function` that we use to a custom function to add
markers
port

In [ ]:
from pp.add_pins import add_pins_triangle

c3 = pp.add_pins_container(c, function=add_pins_triangle)
c3

# Cache

To avoid that 2 exact cells are not references of the same cell the `cell` decorator has a
cache where if component has already been build it will return the component
from the cache


In [ ]:
@pp.cell
def wg(length=10, width=1):
    c = pp.Component()
    c.add_polygon([(0, 0), (length, 0), (length, width), (0, width)], layer=(1, 0))
    print("calling wg function")
    return c

In [ ]:
wg1 = wg()  # cell builds a waveguide
print(wg1)

In [ ]:
wg2 = wg()  # cell returns the same waveguide as before without having to run the function
print(wg2) # notice that they have the same uuid (unique identifier)

In [ ]:
wg2

Lets say that we improve the code of the waveguide function in a jupyter notebook like this one.

I use Vim/VsCode/Pycharm when building new cells in python but some people love jupyter notebooks.

In [ ]:
@pp.cell
def wg(length=10, width=1, layer=(2, 0)):
    """ Adding layer as a function parameter and using layer (2, 0) as default"""
    c = pp.Component()
    c.add_polygon([(0, 0), (length, 0), (length, width), (0, width)], layer=layer)
    print("calling wg function")
    return c

In [ ]:
wg3 = wg()  # Error! cell returns the same waveguide as before! This waveguide should be in layer (2, 0)
print(wg3)
pp.qp(wg3)

In [ ]:
wg4 = wg(cache=False) # Forces a rebuild of the cache. This is very helpful changing function `wg` in a jupyternotebook. Even though I do not reccommend doing layout in jupyter notebooks.
print(wg4)
pp.qp(wg4)
# Note waveguide has different layer now and different uid

In [ ]:
pp.clear_cache() # another option is to clear all the cache
wg5 = wg()
print(wg5)
pp.qp(wg5)
# Note waveguide has different layer now and different uid